In [54]:
import pandas as pd
df1 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Haplotypes_1_12_2023/main_new_data.xlsx')
df1

import pandas as pd

vcf = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/vcf_files_all/KHAIGPRX1351_final.vcf', comment= '#', sep = '\t', header=None, low_memory=False)
vcf.columns = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']

sample_cols = vcf['SAMPLE'].str.split(':', expand=True)
sample_cols.columns = ['GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL', 'RBQ', 'ABQ', 'RDF', 'RDR', 'ADF', 'ADR']

# Assign the values to the newly created columns
vcf = pd.concat([vcf, sample_cols], axis=1)
vcf['HET'] = vcf['INFO'].str.extract(r'HET=(\d)')
vcf['HOM'] = vcf['INFO'].str.extract(r'HOM=(\d)')

# Create a new column 'Zygosity' based on conditions
vcf['Zygosity'] = ''

vcf.loc[vcf['HOM'] == '1', 'Zygosity'] = 'Homozygous'
vcf.loc[vcf['HET'] == '1', 'Zygosity'] = 'Heterozygous'
vcf = vcf[['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL','RDF', 'RDR', 'ADF', 'ADR', 'Zygosity']]
vcf

df = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/KAPA HyperExome_hg38_capture_targets (1).bed', sep = '\t', header = None, error_bad_lines=False)
df.columns = ['chromosome', 'Start_pos', 'End_pos', 'INFO']
df['Extended_Start_pos'] = df['Start_pos'] - 20
df['Extended_End_pos'] = df['End_pos'] + 20
df = df[['chromosome', 'Extended_Start_pos', 'Extended_End_pos']]

# Step 1: Create a dictionary from the df DataFrame
chromosome_dict = {}
for _, row in df.iterrows():
    chromosome = row['chromosome']
    start_pos = row['Extended_Start_pos']
    end_pos = row['Extended_End_pos']
    if chromosome not in chromosome_dict:
        chromosome_dict[chromosome] = []
    chromosome_dict[chromosome].append((start_pos, end_pos))

# Step 2: Define a function to check coverage
def check_coverage(row):
    pos = row['POS']
    chromosome = row['CHROM']
    if chromosome in chromosome_dict:
        ranges = chromosome_dict[chromosome]
        for start, end in ranges:
            if start <= pos <= end:
                return 'Covered'
    return 'Not_Covered'

# Step 3: Apply the function to create the new column in data
vcf['Covered/Not_Covered'] = vcf.apply(check_coverage, axis=1)
vcf

merged = pd.merge(vcf, df1, on = ['CHROM', 'POS', 'REF', 'ALT'], how = 'inner', sort=False)
merged

df_1 = vcf.copy()
df_2 = merged.copy()

df_2['Gene'] = df_2['Haplotype'].str.split('*').str[0]
df_2

df_2 = df_2.drop_duplicates(subset=['Gene', 'Haplotype'], keep='first').reset_index(drop=True)
df_2['Haplotype_new'] = "*" + df_2['Haplotype'].str.split('*').str[1]
df_2

import pandas as pd
from itertools import combinations_with_replacement
# Function to generate combinations
def get_combinations(group):
    return list(combinations_with_replacement(group, 2))

# Apply the function to each group of rsID
result = df_2.groupby('Gene')['Haplotype'].apply(get_combinations).explode().reset_index(name='Diplotype')

# Extract the relevant portions of the haplotypes and keep the asterisk (*)
result['Diplotype'] = result['Diplotype'].apply(lambda x: '/'.join([hap.replace(hap.split('*')[0], '') if len(hap.split('*')) > 1 else hap for hap in x]))

# Filter out reversed combinations
result = result[result['Diplotype'].apply(lambda x: x[0] != x[2])]

# Drop duplicates
result = result.drop_duplicates(subset=['Gene', 'Diplotype'], keep='first').reset_index(drop=True)

# Print the result
result

df2 = df_2.copy()
df2 = df2.rename(columns={'Haplotype_new': 'Diplotype', 'Covered/Not_Covered':'Coverage'})
df2 = df2[['Gene', 'Diplotype', 'Coverage', 'Zygosity']]
df2

# Create an empty 'Coverage' column in dataset1
result['Coverage'] = ''
result['Zygosity'] = ''

# Iterate over rows in dataset1
for index, row in result.iterrows():
    gene = row['Gene']
    diplotype_parts = row['Diplotype'].split('/')
    
    # Check if each part of the diplotype is covered in dataset2
    coverages = []
    zygosities = []
    for part in diplotype_parts:
        coverage = df2.loc[(df2['Gene'] == gene) & (df2['Diplotype'] == part), 'Coverage'].values
        zygosity = df2.loc[(df2['Gene'] == gene) & (df2['Diplotype'] == part), 'Zygosity'].values
        
        coverages.append(coverage[0] if coverage else 'Not Covered')
        zygosities.append(zygosity[0] if zygosity else 'Not Specified')

    # Set the 'Coverage' and 'Zygosity' column values in dataset1
    result.at[index, 'Coverage'] = '/'.join(coverages)
    result.at[index, 'Zygosity'] = '/'.join(zygosities)

result[['Gene', 'Diplotype', 'Coverage', 'Zygosity']]
result

data = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Diplotype_phenotype.xlsx')
data

result.Gene.value_counts()

df_merged = pd.merge(result, data, on = ['Gene', 'Diplotype'], how = 'left', sort = False)
df_merged

df_1 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Star_allele_function.xlsx')
df_1

# Function to map Diplotype to Function
def map_diplotype_to_function(row):
    gene = row['Gene']
    diplotype_parts = row['Diplotype'].split('/')
    
    function_parts = [
        df_1.loc[(df_1['Gene'] == gene) & (df_1['Allele'] == part), 'Function'].values[0] 
        if not df_1.loc[(df_1['Gene'] == gene) & (df_1['Allele'] == part), 'Function'].empty 
        else '-' for part in diplotype_parts
    ]
    
    return '/'.join(function_parts)

# Apply the mapping function to create a new column
df_merged['New_Column'] = df_merged.apply(map_diplotype_to_function, axis=1)

# Display the result
result_df = df_merged[['Gene', 'Diplotype', 'Coverage','New_Column', 'Zygosity', 'Coded Diplotype/Phenotype Summary']]
result_df

,Haplotype,rsID,CHROM,POS,REF,ALT,Genomic_POS,Type
0,CYP2A6*2,rs1801272,chr19,40848628.0,A,T,g.40848628A>T,Haplo
1,CYP2A6*7,rs5031016,chr19,40843869.0,A,G,g.40843869A>G,Haplo
2,CYP2A6*7,rs59322119,chr19,40843749.0,CC,C,g.40843750del,Haplo
3,CYP2A6*7,rs28399469,chr19,40843746.0,T,C,g.40843746T>C,Haplo
4,CYP2A6*7,rs28399470,chr19,40843742.0,G,A,g.40843742G>A,Haplo
...,...,...,...,...,...,...,...,...
724,G6PD*Musashino,NaN,chrX,154536019.0,G,A,g.154536019G>A,Haplo
725,G6PD*Kambos,NaN,chrX,154536151.0,G,A,g.154536151G>A,Haplo
726,"G6PD*Kamiube, Keelung",rs1557229502,chrX,154532258.0,G,A,g.154532258G>A,Haplo
727,G6PD*Lages,NaN,chrX,154546116.0,C,T,g.154546116C>T,Haplo


In [52]:
import pandas as pd
df1 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Haplotypes_1_12_2023/main_new_data.xlsx')
df1

import pandas as pd

vcf = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/vcf_files_all/KHAIGPRX1351_final.vcf', comment= '#', sep = '\t', header=None, low_memory=False)
vcf.columns = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']

sample_cols = vcf['SAMPLE'].str.split(':', expand=True)
sample_cols.columns = ['GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL', 'RBQ', 'ABQ', 'RDF', 'RDR', 'ADF', 'ADR']

# Assign the values to the newly created columns
vcf = pd.concat([vcf, sample_cols], axis=1)
vcf['HET'] = vcf['INFO'].str.extract(r'HET=(\d)')
vcf['HOM'] = vcf['INFO'].str.extract(r'HOM=(\d)')

# Create a new column 'Zygosity' based on conditions
vcf['Zygosity'] = ''

vcf.loc[vcf['HOM'] == '1', 'Zygosity'] = 'Homozygous'
vcf.loc[vcf['HET'] == '1', 'Zygosity'] = 'Heterozygous'
vcf = vcf[['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL','RDF', 'RDR', 'ADF', 'ADR', 'Zygosity']]

df = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/KAPA HyperExome_hg38_capture_targets (1).bed', sep = '\t', header = None, error_bad_lines=False)
df.columns = ['chromosome', 'Start_pos', 'End_pos', 'INFO']
df['Extended_Start_pos'] = df['Start_pos'] - 20
df['Extended_End_pos'] = df['End_pos'] + 20
df = df[['chromosome', 'Extended_Start_pos', 'Extended_End_pos']]

# Step 1: Create a dictionary from the df DataFrame
chromosome_dict = {}
for _, row in df.iterrows():
    chromosome = row['chromosome']
    start_pos = row['Extended_Start_pos']
    end_pos = row['Extended_End_pos']
    if chromosome not in chromosome_dict:
        chromosome_dict[chromosome] = []
    chromosome_dict[chromosome].append((start_pos, end_pos))

# Step 2: Define a function to check coverage
def check_coverage(row):
    pos = row['POS']
    chromosome = row['CHROM']
    if chromosome in chromosome_dict:
        ranges = chromosome_dict[chromosome]
        for start, end in ranges:
            if start <= pos <= end:
                return 'Covered'
    return 'Not_Covered'

# Step 3: Apply the function to create the new column in data
vcf['Covered/Not_Covered'] = vcf.apply(check_coverage, axis=1)
#vcf = vcf[vcf['Covered/Not_Covered'] == 'Covered']
#vcf["Gene Name"] = vcf["INFO"].str.extract('GENEINFO=(?P<GENEINFO>.+?);')
#vcf['Gene Name'] = vcf['Gene Name'].apply(lambda x: ','.join(set([segment.split(':')[0] for segment in x.split('|')])) if pd.notnull(x) else '')
#vcf.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/KHAIGPRX1331_snp_pos.xlsx', index=False)
vcf

merged = pd.merge(vcf, df1, on = ['CHROM', 'POS', 'REF', 'ALT'], how = 'inner', sort=False)
merged

df_1 = vcf.copy()
df_2 = merged.copy()

df_2['Gene'] = df_2['Haplotype'].str.split('*').str[0]
df_2

df_2 = df_2.drop_duplicates(subset=['Gene', 'Haplotype'], keep='first').reset_index(drop=True)
df_2['Haplotype_new'] = "*" + df_2['Haplotype'].str.split('*').str[1]
df_2

import pandas as pd
from itertools import combinations_with_replacement
# Function to generate combinations
def get_combinations(group):
    return list(combinations_with_replacement(group, 2))

# Apply the function to each group of rsID
result = df_2.groupby('Gene')['Haplotype'].apply(get_combinations).explode().reset_index(name='Diplotype')

# Extract the relevant portions of the haplotypes and keep the asterisk (*)
result['Diplotype'] = result['Diplotype'].apply(lambda x: '/'.join([hap.replace(hap.split('*')[0], '') if len(hap.split('*')) > 1 else hap for hap in x]))

# Filter out reversed combinations
result = result[result['Diplotype'].apply(lambda x: x[0] != x[2])]

# Drop duplicates
result = result.drop_duplicates(subset=['Gene', 'Diplotype'], keep='first').reset_index(drop=True)

# Print the result
result

df2 = df_2.copy()
df2 = df2.rename(columns={'Haplotype_new': 'Diplotype', 'Covered/Not_Covered':'Coverage'})
df2 = df2[['Gene', 'Diplotype', 'Coverage', 'Zygosity']]
df2

# Create an empty 'Coverage' column in dataset1
result['Coverage'] = ''

# Iterate over rows in dataset1
for index, row in result.iterrows():
    gene = row['Gene']
    diplotype_parts = row['Diplotype'].split('/')
    
    # Check if each part of the diplotype is covered in dataset2
    coverages = []
    for part in diplotype_parts:
        coverage = df2.loc[(df2['Gene'] == gene) & (df2['Diplotype'] == part), 'Coverage'].values
        coverages.append(coverage[0] if coverage else 'Not Covered')

    # Set the 'Coverage' column value in dataset1
    result.at[index, 'Coverage'] = '/'.join(coverages)
    
result

data = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Diplotype_phenotype.xlsx')
data

result.Gene.value_counts()

df_merged = pd.merge(result, data, on = ['Gene', 'Diplotype'], how = 'left', sort = False)
df_merged

df_1 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Star_allele_function.xlsx')
df_1

# Function to map Diplotype to Function
def map_diplotype_to_function(row):
    gene = row['Gene']
    diplotype_parts = row['Diplotype'].split('/')
    
    function_parts = [
        df_1.loc[(df_1['Gene'] == gene) & (df_1['Allele'] == part), 'Function'].values[0] 
        if not df_1.loc[(df_1['Gene'] == gene) & (df_1['Allele'] == part), 'Function'].empty 
        else '-' for part in diplotype_parts
    ]
    
    return '/'.join(function_parts)

# Apply the mapping function to create a new column
df_merged['New_Column'] = df_merged.apply(map_diplotype_to_function, axis=1)

# Display the result
result_df = df_merged[['Gene', 'Diplotype', 'Coverage','New_Column', 'Coded Diplotype/Phenotype Summary']]
result_df

,Gene,Diplotype,Coverage,New_Column,Coded Diplotype/Phenotype Summary
0,CYP2A6,*7/*7,Not_Covered/Not_Covered,-/-,NaN
1,CYP2A6,*7/*9,Not_Covered/Not_Covered,-/-,NaN
2,CYP2A6,*7/*10,Not_Covered/Not_Covered,-/-,NaN
3,CYP2A6,*7/*19,Not_Covered/Not_Covered,-/-,NaN
4,CYP2A6,*7/*23,Not_Covered/Not_Covered,-/-,NaN
...,...,...,...,...,...
156,NAT2,*12A/*12A,Covered/Covered,-/-,NaN
157,NUDT15,*2/*2,Covered/Covered,-/-,NaN
158,NUDT15,*2/*3,Covered/Covered,-/-,NaN
159,NUDT15,*3/*3,Covered/Covered,-/-,NaN


In [53]:
vcf = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/vcf_files_all/KHAIGPRX1351_final.vcf', comment= '#', sep = '\t', header=None, low_memory=False)
vcf.columns = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']

sample_cols = vcf['SAMPLE'].str.split(':', expand=True)
sample_cols.columns = ['GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL', 'RBQ', 'ABQ', 'RDF', 'RDR', 'ADF', 'ADR']

# Assign the values to the newly created columns
vcf = pd.concat([vcf, sample_cols], axis=1)
vcf['HET'] = vcf['INFO'].str.extract(r'HET=(\d)')
vcf['HOM'] = vcf['INFO'].str.extract(r'HOM=(\d)')

# Create a new column 'Zygosity' based on conditions
vcf['Zygosity'] = ''

vcf.loc[vcf['HOM'] == '1', 'Zygosity'] = 'Homozygous'
vcf.loc[vcf['HET'] == '1', 'Zygosity'] = 'Heterozygous'
vcf = vcf[['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL','RDF', 'RDR', 'ADF', 'ADR', 'Zygosity']]
vcf

,CHROM,POS,rsID,REF,ALT,QUAL,FILTER,INFO,GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,Zygosity
0,chr1,14653,rs62635297,C,T,.,PASS,ADP=26;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,23,26,26,19,7,26.92%,4.9169E-3,15,4,6,1,Heterozygous
1,chr1,14907,rs6682375,A,G,.,PASS,ADP=40;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,64,40,40,22,18,45%,3.1919E-7,8,14,16,2,Heterozygous
2,chr1,14930,rs6682385,A,G,.,PASS,"ADP=41;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.5178,0....",0/1,73,41,41,21,20,48.78%,4.3347E-8,8,13,15,5,Heterozygous
3,chr1,14937,rs11807796,T,C,.,PASS,ADP=39;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=WASH...,0/1,36,39,39,28,11,28.21%,2.1547E-4,20,8,2,9,Heterozygous
4,chr1,16495,rs3210724,G,C,.,PASS,ADP=36;WT=0;HET=1;HOM=0;NC=0;ASP;G5;G5A;GENEIN...,0/1,110,36,36,10,26,72.22%,9.2118E-12,10,0,25,1,Heterozygous
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74651,chrY,57209365,.,T,C,.,PASS,ADP=25;WT=0;HET=1;HOM=0;NC=0;CSQ=C|intron_vari...,0/1,30,25,25,16,9,36%,8.1542E-4,13,3,2,7,Heterozygous
74652,chrY,57209613,.,A,G,.,PASS,ADP=25;WT=0;HET=0;HOM=1;NC=0;CSQ=G|non_coding_...,1/1,115,25,25,2,23,92%,2.7767E-12,0,2,18,5,Homozygous
74653,chrY,57209764,.,C,T,.,PASS,ADP=14;WT=0;HET=0;HOM=1;NC=0;CSQ=T|intron_vari...,1/1,55,14,14,2,12,85.71%,2.9913E-6,2,0,2,10,Homozygous
74654,chrY,57211706,.,G,C,.,PASS,ADP=21;WT=0;HET=1;HOM=0;NC=0;CSQ=C|intron_vari...,0/1,23,21,21,14,7,33.33%,4.3101E-3,7,7,5,2,Heterozygous
